In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from isabelle_connector.isabelle_connector import IsabelleConnector

In [ ]:
isabelle = IsabelleConnector(
    name="extract_transitions", working_directory=".", debug=True
)

In [4]:
from argparse import Namespace

from isabelle_connector.config import HOL_DIR, PROJ_ROOT

configs = Namespace(
    # dataset_name="yalhessi/lemexp",
    # dataset_config="hol-thms",
    root_dir=HOL_DIR,
    imports=[
        PROJ_ROOT / "isabelle-thys/ExtractLemmas",
        PROJ_ROOT / "isabelle-thys/RoughSpec",
    ],
    # upload=True,
)

In [5]:
from isabelle_connector.data_extraction import transitions_theory
from isabelle_connector.utils import get_theory, list_theory_files


thy_files = list_theory_files(configs.root_dir)
src_thys = [get_theory(theory_file, configs.root_dir) for theory_file in thy_files]
transition_thys = [transitions_theory(thy, configs) for thy in src_thys]


In [ ]:
data, errs = isabelle.use_theories(transition_thys, batch_size=1000, use_cache=False)

In [ ]:
from isabelle_connector.data_extraction import template_and_type_extraction_theory


extraction_thys = [
    template_and_type_extraction_theory(thy, configs) for thy in src_thys
]

In [ ]:
data, errs = isabelle.use_theories(extraction_thys, batch_size=1)

In [ ]:
failed_thys = [thy for thy in extraction_thys if errs[thy]]
print(f"Length of failed theories: {len(failed_thys)}")

In [ ]:
silent_thys = [thy for thy in extraction_thys if not errs[thy] and not data[thy]]
print(f"Length of silent theories: {len(silent_thys)}")